# Implementing a 3D mesh simplification algorithm
### A long and bug filled story

#### Why I did it?


I have always fancied computer graphics. Nevertheless, my journey in university led to me to specialize in Mathematical logic.

After starting to work for a company which specializes in 3D rendering, I got inspired to see if I can make any contributions for easing the manual labour work of artists like retopology of a 3D model (includes mesh simplification) which in cases of high-poly meshes may take a lot of work days. The main goal in the future is to see how AI can help to speed up the process. For now, I wanted to start from one of the basic mash simplification algorithms and implement it.

I first found the thesis of [Elena Ovreiu (2012)](https://theses.hal.science/tel-01224848/file/these.pdf) which mentioned this paper of [Garland and Heckbert (1997)](https://www.cs.cmu.edu/~garland/Papers/quadrics.pdf) as being the base for their work. I could have gone even way back the 90-ties papers published on this topic, but I thought that [Garland and Heckbert (1997)](https://www.cs.cmu.edu/~garland/Papers/quadrics.pdf) is a good place to start and build.

#### What is the algorithm explained in [Garland and Heckbert (1997)](https://www.cs.cmu.edu/~garland/Papers/quadrics.pdf) in short?

The algorithm aims to produce high-quality approximations of polygonal models by simplifying the surface using iterative contractions of vertex pairs. The process utilizes quadric matrices (more on them later) to maintain surface error approximations, enabling the joining of unconnected regions and supporting non-manifold models. Non-manifold models have complex topology with edges or vertices shared by more than two faces or having intersecting elements, allowing for more intricate structures, whereas manifold meshes have a simple, clean topology with each edge shared by exactly two faces, resembling a smooth surface.

#### Key Components of the Algorithm:

1. Model Assumption:
    * Focuses on the simplification of polygonal models, particularly those consisting of triangles **(in my case I triangulate if needed)**.

2. Iterative Pair Contraction:
    * Simplifies the model by contracting vertex pairs iteratively, generalizing the edge contraction method.
    * Contraction allows for the joining of unconnected regions (aggregation) and supports non-manifold models.

3. Error Approximation with Quadric Matrices:
    * Uses quadric matrices to maintain and approximate geometric errors at each vertex during simplification. A quadric matrix is a symmetric 4x4 matrix that represents a quadratic form. In the context of this algorithm, it is used to approximate the error at each vertex of a 3D model during the simplification process. The idea is to keep track of how much the geometry of the model deviates from the original as vertices are contracted. The level surfaces of the quadric error function $v^TQv$ are quadric surfaces, which can be ellipsoids, paraboloids, or hyperboloids depending on the specific values in the matrix $Q$. These surfaces help visualize how the error is distributed around a vertex. For example, in a mostly planar region (a flat, two-dimensional surface area within a three-dimensional space), the quadric surfaces might be flat ellipsoids, indicating that moving the vertex within this ellipsoid would not significantly increase the error.

4. Contraction Process:
    * Contracts the vertex pairs $(v_1, v_2)$ to a new position $\bar v$ and updates the model.
    * Optimal position $\bar v$ is determined by minimizing the error quadratic form.
    * Valid pairs are determined based on whether they are edges or within a threshold distance $t$.

5. Algorithm Workflow:
    * Compute initial $Q$ matrices for all vertices.
    * Select valid pairs and compute contraction targets.
    * Use a priority queue to iteratively contract the least cost pairs until the desired level of simplification is achieved.

6. Boundary Preservation and Mesh Inversion Prevention:
    * Preserves boundaries by introducing penalty quadrics for discontinuity edges.
    * Prevents mesh inversion by penalizing or disallowing contractions that cause normal flips in neighbouring faces.

#### Pros and Cons

##### Pros:
1. **Efficiency**:
   - Rapid simplification of complex models. **In the case of my implementation, it is not at all as fast as what they claim in the paper, but I am content with being able to have my own version of it.**
   - Compact error approximation requires only 10 floating point numbers per vertex.

2. **Quality**:
   - High fidelity approximations are maintained, preserving primary features even after significant simplification.
   - Optimal vertex placement further reduces approximation errors.

3. **Generality**:
   - Capable of joining unconnected regions (aggregation) and supporting non-manifold models.
   - Suitable for various applications where maintaining topology is not crucial.

##### Cons:
1. **Error Metric Limitation**:
   - Error metric as a distance to a set of planes is less reliable for non-local neighbourhoods, leading to potential inaccuracies in certain cases.
   - The algorithm may not perform well when significant aggregation is required due to the implicit nature of information in quadrics.

2. **Complexity in Handling Surface Properties**:
   - Extending the algorithm to handle surface properties like colour involves additional complexity and storage requirements.

3. **Inability to Remove Defunct Faces**:
   - Difficulty in determining and reliably removing defunct faces (faces that no longer serve a purpose or contribute to the geometry of the model) after aggregation may affect the quality of simplification in some scenarios.

#### Some results on two dinosaur meshes (Spinosaurus and Tyrannosaurus)

##### Spinosaurus no simplification
<img src="images/spinoFull.png" alt="Spinosaurus no simplification" width="45%">

##### Spinosaurus 50% simplification
<img src="images/spino50P.png" alt="Spinosaurus 50% simplification" width="45%">

##### Spinosaurus 90% simplification
<img src="images/spino90P.png" alt="Spinosaurus 90% simplification" width="45%">

##### Spinosaurus 90% simplified overlap with high-poly mesh                                            
<img src="images/overlapSpino10%25WithHighPoly.png" alt="Spinosaurus overlap meshes" width="45%">

##### Tyrannosaurus no simplification
<img src="images/trexFull.png" alt="Tyrannosaurus no simplification" width="45%">

##### Tyrannosaurus 50% simplification
<img src="images/trex50P.png" alt="Tyrannosaurus 50% simplification" width="45%">

##### Tyrannosaurus 90% simplification
<img src="images/trex90P.png" alt="Tyrannosaurus 90% simplification" width="45%">

### P.S:
The nasty bug was in the calculation of the cost o contraction for a pair. Instead of creating a new matrix with which to calculate the optimal position of the vertex, I overwrote the old one (sum of the quadric matrices of the two vertices in the pair). That way the cost for contracting a pair was always around 1 and so the algorithm got delivered a mostly intact mesh which had one region of it totally destroyed (made into a cone).